# 使用keras实现单个预训练模型训练

1 使用 Xception

2 冻结

3 训练

### 1 import libs

In [1]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

Using TensorFlow backend.


### 2 build pretrained model

In [2]:
input_tensor = Input((299, 299, 3))
x = Lambda(xception.preprocess_input)(input_tensor)

base_model = Xception(input_tensor=x, weights='imagenet', include_top=False)
m_out = base_model.output
p_out = GlobalAveragePooling2D()(m_out)
p_out = Dropout(0.5)(p_out)
predictions = Dense(5, activation='softmax')(p_out)

for layer in base_model.layers:  
     layer.trainable = False

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='Adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         lambda_1[0][0]                   
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_con

### 3 build param

In [3]:
batch_size = 32
epochs = 1

total_imgs_num = 3309 + 373
train_imgs_num = 3309
valid_imgs_num = 373

img_size = (299, 299)

### 4 build data & train

In [4]:
img_gen_t = ImageDataGenerator()
train_generator = img_gen_t.flow_from_directory("../flower_photos/train-ready", img_size, shuffle=True, batch_size=batch_size)

img_gen_v = ImageDataGenerator()
valid_generator = img_gen_v.flow_from_directory("../flower_photos/valid-ready", img_size, shuffle=True, batch_size=batch_size)


model.fit_generator(generator=train_generator, steps_per_epoch=train_imgs_num//batch_size, epochs=epochs, verbose=1,
                   validation_data=valid_generator, validation_steps=valid_imgs_num//batch_size)

model.save_weights("Xception_first_train.h5")

Found 3303 images belonging to 5 classes.
Found 367 images belonging to 5 classes.
Epoch 1/1
104/103 [==============================] - 115s 1s/step - loss: 1.1711 - acc: 0.5894 - val_loss: 0.8311 - val_acc: 0.7548


### 5 finetune

In [5]:
input_tensor = Input((299, 299, 3))
x = Lambda(xception.preprocess_input)(input_tensor)

base_model = Xception(input_tensor=x, weights=None, include_top=False)
m_out = base_model.output
p_out = GlobalAveragePooling2D()(m_out)
p_out = Dropout(0.5)(p_out)
predictions = Dense(5, activation='softmax')(p_out)
model = Model(inputs=base_model.input, outputs=predictions)
model.load_weights('Xception_first_train.h5')
model.compile(optimizer='Adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 299, 299, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         lambda_2[0][0]                   
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_con

In [6]:
batch_size = 24
epochs = 10

total_imgs_num = 3309 + 373
train_imgs_num = 3309
valid_imgs_num = 373

img_size = (299, 299)

img_gen_t = ImageDataGenerator()
train_generator = img_gen_t.flow_from_directory("../flower_photos/train-ready", img_size, shuffle=True, batch_size=batch_size)

img_gen_v = ImageDataGenerator()
valid_generator = img_gen_v.flow_from_directory("../flower_photos/valid-ready", img_size, shuffle=True, batch_size=batch_size)


model.fit_generator(generator=train_generator, steps_per_epoch=train_imgs_num//batch_size, epochs=epochs, verbose=1,
                   validation_data=valid_generator, validation_steps=valid_imgs_num//batch_size)

model.save_weights("Xception_finetune.h5")

Found 3303 images belonging to 5 classes.
Found 367 images belonging to 5 classes.
Epoch 1/10
138/137 [==============================] - 65s 473ms/step - loss: 0.3064 - acc: 0.8991 - val_loss: 0.1442 - val_acc: 0.9401
Epoch 2/10
138/137 [==============================] - 60s 432ms/step - loss: 0.0617 - acc: 0.9821 - val_loss: 0.1309 - val_acc: 0.9482
Epoch 3/10
138/137 [==============================] - 60s 432ms/step - loss: 0.0283 - acc: 0.9923 - val_loss: 0.1550 - val_acc: 0.9482
Epoch 4/10
138/137 [==============================] - 59s 431ms/step - loss: 0.0180 - acc: 0.9955 - val_loss: 0.1435 - val_acc: 0.9455
Epoch 5/10
138/137 [==============================] - 60s 432ms/step - loss: 0.0094 - acc: 0.9983 - val_loss: 0.1660 - val_acc: 0.9564
Epoch 6/10
138/137 [==============================] - 60s 431ms/step - loss: 0.0095 - acc: 0.9973 - val_loss: 0.1586 - val_acc: 0.9619
Epoch 7/10
138/137 [==============================] - 60s 431ms/step - loss: 0.0091 - acc: 0.9973 - val_los

### 5 load weights & predict

In [43]:
input_tensor = Input((299, 299, 3))
x = Lambda(xception.preprocess_input)(input_tensor)

base_model = Xception(input_tensor=x, weights=None, include_top=False)
m_out = base_model.output
p_out = GlobalAveragePooling2D()(m_out)
p_out = Dropout(1.0)(p_out)
fc_out = Dense(1024, activation='relu')(p_out)
fc_out = Dropout(1.0)(fc_out)
predictions = Dense(3, activation='softmax')(fc_out)
model = Model(inputs=base_model.input, outputs=predictions)
model.load_weights('Xception.h5')

In [44]:
img_gen_s = ImageDataGenerator()
submit_generator = img_gen_t.flow_from_directory("../dataset/test", img_size, shuffle=False, batch_size=2)

predictions = []
for i, batch_data in enumerate(submit_generator):
    if (i > 1):
        break
    batch_img_data = np.array(batch_data[0])
    predictions.extend(model.predict_on_batch(batch_img_data))
    print(i)
    
print(predictions)

Found 14 images belonging to 3 classes.
0
1
[array([ 0.30176267,  0.35524741,  0.34298989], dtype=float32), array([ 0.28475785,  0.34978592,  0.36545616], dtype=float32), array([ 0.3018229 ,  0.35525236,  0.34292471], dtype=float32), array([ 0.29986629,  0.35433695,  0.34579682], dtype=float32)]


In [45]:
print(np.array(predictions))

[[ 0.30176267  0.35524741  0.34298989]
 [ 0.28475785  0.34978592  0.36545616]
 [ 0.3018229   0.35525236  0.34292471]
 [ 0.29986629  0.35433695  0.34579682]]


### 6 write predictions 2 CSV